In [1]:
import os

from langchain.agents import Tool
from langchain.memory import ChatMessageHistory, ConversationBufferWindowMemory
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.agents import AgentExecutor
from langchain import hub
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datetime import datetime
hoje = datetime.today()

data_formatada = hoje.strftime("%Y/%m/%d")
# Pega o dia da semana em inglês
dia_da_semana_ingles = hoje.strftime("%A")

# Dicionário para mapear os dias da semana de inglês para português
dias_da_semana_pt = {
    "Monday": "segunda-feira",
    "Tuesday": "terça-feira",
    "Wednesday": "quarta-feira",
    "Thursday": "quinta-feira",
    "Friday": "sexta-feira",
    "Saturday": "sábado",
    "Sunday": "domingo"
}

# Traduz o dia da semana para português
dia_da_semana_pt = dias_da_semana_pt.get(dia_da_semana_ingles, dia_da_semana_ingles)

# Formata a data para utilizar no prompt
data_atual = f"{dia_da_semana_pt}, {data_formatada}"

In [3]:
#(https://makersuite.google.com/)
from dotenv import load_dotenv

load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')

In [4]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    convert_system_message_to_human=True,
    handle_parsing_errors=True,
    temperature=0.6,
    max_tokens= 1000,
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    },
)

In [5]:
def transfer_to_calendar_agent(input_str):
    return calendar_agent_executor.invoke({"input": input_str})
def transfer_to_travel_agent(input_str):
    return travel_agent_executor.invoke({"input": input_str})

In [6]:
from calendar_tools import extract_event_parameters, insert_calendar_event
print(insert_calendar_event("""
**Dia 11 de Janeiro de 2025 (Sexta-feira):**

* **Noite (18:00 - 22:00):** Jantar em um restaurante com culinária potiguar.  Pesquise opções na região de Ponta Negra ou no centro de Natal. Após o jantar, aproveite para um passeio noturno pela orla de Ponta Negra.
"""))

calendar v3 service created successfully


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


{'kind': 'calendar#event', 'etag': '"3473241386074000"', 'id': '1cuh83m8an2mli12rirb9nh7cc', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=MWN1aDgzbThhbjJtbGkxMnJpcmI5bmg3Y2MgdGFsZXMudi5tLmFsdmVzQG0', 'created': '2025-01-11T18:38:13.000Z', 'updated': '2025-01-11T18:38:13.037Z', 'summary': 'Jantar e passeio em Natal', 'description': 'Jantar em um restaurante com culinária potiguar. Pesquise opções na região de Ponta Negra ou no centro de Natal. Após o jantar, passeio noturno pela orla de Ponta Negra.', 'location': 'Restaurante em Ponta Negra ou Centro de Natal + Orla de Ponta Negra', 'creator': {'email': 'tales.v.m.alves@gmail.com', 'self': True}, 'organizer': {'email': 'tales.v.m.alves@gmail.com', 'self': True}, 'start': {'dateTime': '2025-01-11T18:00:00-03:00', 'timeZone': 'America/Fortaleza'}, 'end': {'dateTime': '2025-01-11T22:00:00-03:00', 'timeZone': 'America/Fortaleza'}, 'iCalUID': '1cuh83m8an2mli12rirb9nh7cc@google.com', 'sequence': 0, 'reminders'

In [7]:
from planing_tools import weatherapi_forecast_periods, query_rag
ddg_search = DuckDuckGoSearchAPIWrapper()

travel_planing_tools = [
    Tool(
        name="DuckDuckGo Search",
        func=ddg_search.run,
        description="""Essa ferramenta DEVE ser utilizada para buscar eventos relevantes no período fornecido pelo usuário. 
        Ela é útil para obter informações sobre eventos ou atividades especiais que estão acontecendo na cidade de destino nas datas que o usuário informou. 
        O modelo deve usá-la para complementar as sugestões de atividades."""
    ),
    Tool(
        name="Weather Forecast",
        func=weatherapi_forecast_periods,
        description="""Essa ferramenta DEVE ser usada *antes* de gerar o roteiro turístico, após o modelo coletar todas as informações necessárias do usuário, incluindo as datas exatas. 
    O modelo deve consultar o clima separadamente para cada dia do período informado, garantindo que as atividades do roteiro sejam planejadas de acordo com as condições climáticas. 
    A previsão do tempo é um passo obrigatório quando é um fator relevante para o planejamento."""
    ),
    Tool(
        name="Query RAG",
        func=query_rag,
        description="""Esta ferramenta deve ser usada quando o modelo souber a cidade de destino e os interesses do usuário, com o objetivo de fornecer informações sobre pontos turísticos e atrações que se alinham com esses interesses. 
        O modelo deve utilizar essa ferramenta para sugerir atividades e lugares específicos a visitar, baseados na cidade e nos interesses fornecidos."""
    ),
    Tool(
        name="Calendar Agent",
        func=transfer_to_calendar_agent,
        description="Esse agente lida com tudo relacionado ao calendário. As mensagens enviadas a ele devem estar em Português."
    ),
]

In [8]:
from calendar_tools import list_calendar_list, list_calendar_events, insert_calendar_event, create_calendar

google_calendar_tools = [
    Tool(
        name="Create Calendar List",
        func=create_calendar,
        description="""
        Use a função create_calendar_list para criar um novo calendário.
        Forneça o resumo (título) do calendário que você deseja criar.
        - Exemplo de uso: 'Meu Calendário de Viagens'
        - Essa função vai criar um novo calendário com o resumo fornecido.
        """
    ),
    Tool(
        name="List Calendar List",
        func=list_calendar_list,
        description="""
        Use a função list_calendar_list para obter a lista dos seus calendários disponíveis no Google Calendar.
        Se você quiser limitar o número de calendários retornados, forneça o número máximo desejado.
        - Exemplo de uso: Forneça o número máximo de calendários a serem listados, como 50. Se não quiser um limite, apenas diga para listar 200 o máximo por padrão.
        """
    ),
    Tool(
        name="List Calendar Events",
        func=list_calendar_events,
        description="""
        Use a função list_calendar_events para obter a lista de eventos de um calendário específico.
        Você precisa fornecer o ID do calendário e, opcionalmente, o número máximo de eventos a serem listados.
        - Exemplo de uso para o calendário principal: forneça 'primary' como o ID do calendário.
        - Exemplo de uso com limite: forneça o ID do calendário e o número máximo de eventos, como 'primary', 20.
        - Para listar eventos de um calendário específico, primeiro liste seus calendários para encontrar o ID correto.
        """
    ),
    Tool(
    name="Insert Calendar Event",
    func=insert_calendar_event,
    description="""
        Use a função Insert Calendar Event para adicionar **um único evento** a um calendário Google.
        Esta função deve ser chamada **uma vez para cada evento** que precisa ser agendado.

        A entrada para esta função deve ser uma descrição clara e completa de **um único evento**
        em linguagem natural, contendo todas as informações necessárias para agendá-lo individualmente.

        Se o usuário pediu para criar um novo calendário, você DEVE utilizar o calendar_id desse novo calendário para a inserção dos eventos. 
        
        **Formato de entrada esperado:**

        **Descrição em Linguagem Natural (obrigatório):**
        Forneça uma descrição textual detalhada para **um único evento**. Inclua o calendar_id (se fornecido pelo usuário), o que será feito,
        quando (data e hora específicas), onde (local exato) e quaisquer detalhes adicionais relevantes
        para esse evento em particular.

        **Exemplos de descrições de eventos individuais:**
        - "Agendar almoço com a Maria na sexta-feira ao meio-dia no restaurante X."
        - "Criar um compromisso para pagar as contas no dia 10 de janeiro às 9h da manhã."
        - "Bloquear minha agenda para foco no projeto Y na próxima segunda-feira, das 14h às 17h, na minha mesa."

        **Como usar esta função corretamente:**

        1. **Identifique cada evento individualmente:** Se a intenção é agendar múltiplos eventos,
           trate cada um como uma chamada separada para esta função.
        2. **Obtenha o ID do calendário (se necessário):** Se o usuário mencionar um nome de
           calendário específico, use a função 'list_calendar_list' **primeiro** para obter
           o 'calendar_id' correspondente.
        3. **Chame esta função com a descrição de um único evento:** Forneça a descrição completa
           em linguagem natural para o evento específico que você deseja agendar.

        **Informações essenciais para cada descrição de evento:**
        - **O que:** Uma descrição concisa do evento (ex: "Reunião de equipe", "Consulta médica").
        - **Quando:** A data e hora de início e fim **precisas** do evento.
        - **Onde:** O local **específico** do evento.
        - **Fuso Horário (opcional):** Se aplicável, mencione o fuso horário se for diferente
          de "America/Fortaleza".

        **Exemplo de uso**

        *Usuário: Agendar visita ao Aquário Natal sábado das 9h às 12h.*
        *Agente:* (Chama 'Insert Calendar Event' com a Action Input: "Agendar visita ao Aquário Natal sábado das 9h às 12h.")

        **Observação IMPORTANTE:** Esta função **NÃO** deve ser usada para criar múltiplos eventos
        com uma única chamada. Cada chamada deve corresponder a **um único evento**.

        Ao final da execução desta função, você DEVE fornecer ao usuário a confirmação do agendamento
        desse evento específico, incluindo o título, data, hora e link do evento criado (se disponível).
    """
    )
]

In [9]:
travel_planing_prompt = hub.pull("tales/agente_turismo")
google_calendar_prompt = hub.pull("tales/agente_calendario")
# main_agent_prompt = hub.pull("tales/agente_principal")

In [10]:
planing_prompt = travel_planing_prompt.partial(
    tools=render_text_description(travel_planing_tools),
    tool_names=", ".join([t.name for t in travel_planing_tools]),
)
llm_with_stop = llm.bind(stop=["\nObservation"])

In [11]:
calendar_prompt = google_calendar_prompt.partial(
    tools=render_text_description(google_calendar_tools),
    tool_names=", ".join([t.name for t in google_calendar_tools]),
)

In [12]:
history = ChatMessageHistory()
memory = ConversationBufferWindowMemory(k=20, chat_memory=history, memory_key="chat_history")

C:\Users\Tales\AppData\Local\Temp\ipykernel_5320\275711510.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=20, chat_memory=history, memory_key="chat_history")


In [13]:
travel_planing_agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
        "data_atual": lambda x: data_atual,
    }
    | planing_prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

travel_agent_executor = AgentExecutor(agent=travel_planing_agent, tools=travel_planing_tools, verbose=True, memory=memory)

In [14]:
google_calendar_agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
        "data_atual": lambda x: data_atual,
    }
    | calendar_prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

calendar_agent_executor = AgentExecutor(agent=google_calendar_agent, tools=google_calendar_tools, verbose=True, memory=memory)

In [15]:
response = travel_agent_executor.invoke({"input": 'Boa noite'})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: Do I need to use a tool? No
Final Answer: Boa noite! Para começarmos a planejar sua viagem, preciso de algumas informações. Para onde você está planejando viajar?


> Finished chain.
Boa noite! Para começarmos a planejar sua viagem, preciso de algumas informações. Para onde você está planejando viajar?


In [16]:
response = travel_agent_executor.invoke({"input": 'Eu estou planejando visitar Natal no Rio Grande do Norte'})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: Do I need to use a tool? No
Final Answer: Ótimo! Natal é uma excelente escolha.  Quais atividades ou experiências você está buscando nessa viagem? Por exemplo, prefere explorar praias, restaurantes, eventos, museus, história, cultura, vida noturna, natureza, ou algo mais?


> Finished chain.
Ótimo! Natal é uma excelente escolha.  Quais atividades ou experiências você está buscando nessa viagem? Por exemplo, prefere explorar praias, restaurantes, eventos, museus, história, cultura, vida noturna, natureza, ou algo mais?


In [17]:
response = travel_agent_executor.invoke({"input": 'Eu planejo visitar praias e restaurantes regionais'})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Perfeito! Praias e restaurantes regionais em Natal.  Tem um orçamento em mente ou um limite de gastos que gostaria de seguir durante a viagem?
```


> Finished chain.
Perfeito! Praias e restaurantes regionais em Natal.  Tem um orçamento em mente ou um limite de gastos que gostaria de seguir durante a viagem?
```


In [18]:
response = travel_agent_executor.invoke({"input": 'Eu ainda não sei'})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Entendo.  Vamos prosseguir sem um orçamento definido por enquanto. Você segue alguma dieta específica ou tem restrições alimentares, como alergias, que devo levar em consideração?
```


> Finished chain.
Entendo.  Vamos prosseguir sem um orçamento definido por enquanto. Você segue alguma dieta específica ou tem restrições alimentares, como alergias, que devo levar em consideração?
```


In [19]:
response = travel_agent_executor.invoke({"input": 'Não possuo nenhuma restrição alimentar.'})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Ótimo! Sem restrições alimentares.  Quantos dias você pretende passar em Natal?
```


> Finished chain.
Ótimo! Sem restrições alimentares.  Quantos dias você pretende passar em Natal?
```


In [20]:
response = travel_agent_executor.invoke({"input": 'Eu planejo passar 2 dias da sábado dia 11 de janeiro ao domingo dia 12'})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Weather Forecast
Action Input: 2025/01/11, 2025/01/12Previsão para 2025/01/11, 2025/01/12 em Natal:

Manhã:
Hora: 05:00 - Temp: 26.3°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 83%
Hora: 06:00 - Temp: 27.6°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 76%
Hora: 07:00 - Temp: 28.5°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 71%
Hora: 08:00 - Temp: 29.5°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 66%
Hora: 09:00 - Temp: 30.3°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 63%
Hora: 10:00 - Temp: 30.6°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 62%
Hora: 11:00 - Temp: 30.6°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 62%

Tarde:
Hora: 13:00 - Temp: 30.2°C, Condição: Possibilidade de chuva irregula

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Query RAG
Action Input: Atrações turísticas em Natal, Rio Grande do Norte, focadas em praias e restaurantes regionais.ATRAÇÕES
Em Natal, a vida além da 
praia garante aquário, museus, 
parque aquático, cajueiro 
gigante e estádio de futebol
PARQUE  
DAS DUNAS
Um dos grandes 
charmes do gigantesco 
parque é o Bosque dos 
Namorados, que tem 
área de piquenique, 
playground, restaurante e 
centro de visitantes. Vale 
a pena também percorrer 
as trilhas guiadas do 
parque, com vegetação 
exuberante. Embora 
não tenha praia, oferece 
mirantes com lindas vistas 
panorâmicas para o mar.
FOTOS: SHUTTERSTOCK
AQUÁRIO NATAL
Na Praia da Redinha, já no 
caminho para Genipabu, 
possui cerca de 60 
espécies de animais, como 
moreias, peixes de corais, 
cavalos-marinhos, jacarés e 
até mesmo mamíferos, como 
hipopótamos e jaguatiricas. 
Você pode passar a mão 
nas costas dos tubarões e 
chegar perto dos pinguins.

---

PRAIA A PRAIA
A orla urb

...

```tool_code
Thought: Do I need to use a tool? Yes
Action: DuckDuckGo Search
Action Input: eventos em Natal RN 11 e 12 janeiro 2025

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_community\utilities\duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


A programação cultural de janeiro 2025 em Natal será palco de grandes shows e eventos, garantindo diversão para moradores e turistas. Confira os destaques da programação cultural de janeiro 2025 em Natal e prepare-se para aproveitar o melhor da Cidade do Sol! No dia 11 de janeiro de 2025, a partir das 12h (meio-dia), o Rota 360 , realiza o evento Psy Sunset com várias atrações. Mais informações no perfil oficial @sigarota360. Pura Retrô, Valber Fernandes, Dj PL e Pedro Lucas - Pitanga virou Festa! Ed. Beverly Hills. Picanha Go! Conhece algum local e não está listado? Cadastre aqui! Portal sobre tecnologias, eventos, rally, geoprocessamento e confrarias. O verão de 2025 está em pleno vapor na Grande Natal, trazendo uma variedade de eventos culturais para todos os gostos. As praias de Pirangi se destacam com uma programação intensa, incluindo shows de artistas renomados e festas temáticas que prometem agitar as noites litorâneas. Descubra os melhores eventos em Natal, incluindo shows ao 

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Ótimo! Com base nas informações que você me forneceu, preparei um roteiro para sua viagem a Natal nos dias 11 e 12 de janeiro de 2025, focando em praias e restaurantes regionais.  Lembre-se que a previsão do tempo indica chuva, principalmente no dia 11, então é importante levar roupas apropriadas e estar preparado para possíveis alterações nos planos.

**Roteiro de Viagem para Natal (11 e 12 de Janeiro de 2025):**

**Dia 11 de Janeiro (Sábado):**

* **Manhã (8h-12h):**  Considerando a previsão de chuva intensa pela manhã, sugiro começar o dia com um café da manhã reforçado em seu hotel ou em uma cafeteria local.  Em seguida, visite o Aquário de Natal na Praia da Redinha.  A chuva pode dificultar atividades externas, mas o aquário oferece uma opção coberta e interessante.
* **Tarde (13h-17h):**  Se a chuva diminuir, aproveite para visitar a Praia de Ponta Negra e admirar o Morro do Careca (de longe, pois subir é proibido). 

In [21]:
response = travel_agent_executor.invoke({"input": 'Eu quero que você salve os eventos sugeridos no meu calendário google, no novo calendário chamado Janeiro em Natal 2025.'})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Calendar Agent
Action Input: Salve os seguintes eventos no meu Calendário Google, em um novo calendário chamado "Janeiro em Natal 2025":

**Dia 11 de Janeiro (Sábado):**

* **Manhã (8h-12h):** Café da manhã e visita ao Aquário de Natal na Praia da Redinha.
* **Tarde (13h-17h):** Praia de Ponta Negra (se o tempo permitir) ou Parque das Dunas (se chover) e almoço em restaurante regional próximo à Ponta Negra.
* **Noite (18h-23h):** Jantar em restaurante regional e possível show ou evento local (a definir).


**Dia 12 de Janeiro (Domingo):**

* **Manhã (8h-12h):** Café da manhã e visita à Praia do Meio ou Praia dos Artistas (dependendo do tempo) ou Mercado Público da Redinha (se chover).
* **Tarde (13h-17h):** Exploração da Via Costeira e almoço em restaurante regional na Via Costeira.
* **Noite (18h-23h):** Jantar em restaurante de preferência.

Por favor, adicione detalhes sobre os restaurantes assim que eu os pesquisar e te inf

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Create Calendar List
Action Input: Janeiro em Natal 2025{'kind': 'calendar#calendar', 'etag': '"EitsE6nkIJY5B1bHCZi9_eFLkxo"', 'id': '4e6a251b912396387054311b04a383e5688c52f946f8d640143e869cdc529078@group.calendar.google.com', 'summary': 'Janeiro em Natal 2025', 'timeZone': 'UTC', 'conferenceProperties': {'allowedConferenceSolutionTypes': ['hangoutsMeet']}}

...

{'kind': 'calendar#event', 'etag': '"3473241598586000"', 'id': 'ane3ojkf8am2pu6tpkmln5u604', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=YW5lM29qa2Y4YW0ycHU2dHBrbWxuNXU2MDQgNGU2YTI1MWI5MTIzOTYzODcwNTQzMTFiMDRhMzgzZTU2ODhjNTJmOTQ2ZjhkNjQwMTQzZTg2OWNkYzUyOTA3OEBn', 'created': '2025-01-11T18:39:59.000Z', 'updated': '2025-01-11T18:39:59.293Z', 'summary': 'Jantar em restaurante de preferência', 'location': 'A definir, Natal', 'creator': {'email': 'tales.v.m.alves@gmail.com'}, 'organizer': {'email': '4e6a251b912396387054311b04a383e5688c52f946f8d640143e869cdc529078@group.calendar.google.com', 'displayName': 'Janeiro em Natal 2025', 'self': True}, 'start': {'dateTime': '2025-01-12T21:00:00Z', 'timeZone': 'America/Fortaleza'}, 'end': {'dateTime': '2025-01-13T02:00:00Z', 'timeZone': 'America/Fortaleza'}, 'iCalUID': 'ane3ojkf8am2pu6tpkmln5u604@google.com', 'sequence': 0, 'reminders': {'useDefault': True}, 'eventType': 'default'}

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Ótimo! Agendei todos os eventos no seu novo calendário "Janeiro em Natal 2025".  Lembre-se que alguns detalhes, como os nomes dos restaurantes, ainda precisam ser adicionados após você pesquisar e me informar. Assim que tiver essas informações, posso atualizar o calendário com os detalhes completos.  Você pode acessar seu calendário Google para visualizar os eventos agendados.  Precisará de mais alguma coisa?
```


> Finished chain.
{'input': 'Salve os seguintes eventos no meu Calendário Google, em um novo calendário chamado "Janeiro em Natal 2025":\n\n**Dia 11 de Janeiro (Sábado):**\n\n* **Manhã (8h-12h):** Café da manhã e visita ao Aquário de Natal na Praia da Redinha.\n* **Tarde (13h-17h):** Praia de Ponta Negra (se o tempo permitir) ou Parque das Dunas (se chover) e almoço em restaurante regional próximo à Ponta Negra.\n* **Noite (18h-23h):** Jantar em restaurante regional e possível show ou evento local (a definir).\n

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Do I need to use a tool? No
Final Answer: Ótimo! Agendei todos os eventos no seu novo calendário "Janeiro em Natal 2025".  Lembre-se que alguns detalhes, como os nomes dos restaurantes, ainda precisam ser adicionados após você pesquisar e me informar. Assim que tiver essas informações, posso atualizar o calendário com os detalhes completos.  Você pode acessar seu calendário Google para visualizar os eventos agendados.  Precisará de mais alguma coisa?


> Finished chain.
Ótimo! Agendei todos os eventos no seu novo calendário "Janeiro em Natal 2025".  Lembre-se que alguns detalhes, como os nomes dos restaurantes, ainda precisam ser adicionados após você pesquisar e me informar. Assim que tiver essas informações, posso atualizar o calendário com os detalhes completos.  Você pode acessar seu calendário Google para visualizar os eventos agendados.  Precisará de mais alguma coisa?


In [ ]:
print(response["chat_history"])